In [1]:
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

In [2]:
load_kube_config()  # loads local configuration from ~/.kube/config

In [3]:
v1alpha1 = V1alpha1Api()

In [5]:
from argo.workflows.client.models import *

In [38]:
# Create a Workflow with a single task

spec = V1alpha1WorkflowSpec(
    entrypoint="steps", 
    volume_claim_templates=[
        V1PersistentVolumeClaim(
            metadata=V1ObjectMeta(
                name="volume"
            ),
            spec=V1PersistentVolumeClaimSpec(
                access_modes=["ReadWriteOnce"],
                resources=V1ResourceRequirements(
                    requests={
                        "storage": "10Mi"
                    }
                )
            )
        )
    ],
#    volumes=[
#        V1Volume(
#            name="volume",
#            empty_dir={}
#        )
#    ]
    templates=[
        V1alpha1Template(
            name="steps",
            steps=[
                [
                    V1alpha1WorkflowStep(name="catfile", template="catfile")
                ],
                [
                    V1alpha1WorkflowStep(name="show", template="show")
                ]
            ]
            #,
            #archive_location=V1alpha1ArtifactLocation(s3=V1alpha1S3Artifact(
            #    endpoint="minio.default:9000",
            #    insecure=True,
            #    bucket="subjects-data",
            #    access_key_secret=access_key_secret,
            #    secret_key_secret=secret_key_secret,
            #    key="message-stocazzo"
            #))
        ),       
        V1alpha1Template(
            name="catfile", 
            container=V1Container(
                name="catfile", 
                image="ubuntu", 
                command=["bash", "-c", "cd / ; ls -lh ; echo michele > /workdir/stocazzo.txt ; ls /workdir"], 
                args=[],
                volume_mounts=[
                    V1VolumeMount(name="volume", mount_path="/workdir")
                ]
            )
        ),
        V1alpha1Template(
            name="show", 
            container=V1Container(
                name="show", 
                image="ubuntu", 
                command=["bash", "-c", "cd / ; ls -lh / ; ls -lh /workdir ; cat /workdir/stocazzo.txt"], 
                args=[],
                volume_mounts=[
                    V1VolumeMount(name="volume", mount_path="/workdir")
                ]
            )
        )
    ])
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=V1ObjectMeta(generate_name="cat-file-"), 
    spec=spec,
    status=V1alpha1WorkflowStatus())

In [39]:
result = v1alpha1.create_namespaced_workflow('default', workflow)

In [40]:
# The following fragment must be invoked more time, until the workflow is completed its execution

from kubernetes.client.rest import ApiException
from kubernetes import client, config

config.load_kube_config()

wfs = v1alpha1.list_namespaced_workflows(namespace="default")
for job in wfs.items:
    if job.metadata.name == result.metadata.name:
        for pod_name in job.status.nodes.keys():
            if job.status.nodes[pod_name].type == 'Pod':
                try:
                    api_instance = client.CoreV1Api()
                    # Pod created by Argo has init containers, main container and wait container...
                    api_response = api_instance.read_namespaced_pod_log(name=pod_name, container="main", namespace='default')
                    print(f"Log for {pod_name}: {api_response}")
                except ApiException as e:
                    print(f'Found exception in reading the logs {pod_name}')

Log for cat-file-mkj5h-2945398604: 
